In [6]:
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split
import glob
import os


In [12]:
# Prepare the data
seq_length = 10
X = []
y = []

max_seq_length = 0  # Track the maximum sequence length

for action in actions:
    raw_data_files = glob.glob(f"{dataset_path}{action}/raw_*.npy")
    seq_data_files = glob.glob(f"{dataset_path}{action}/seq_*.npy")

    raw_data = np.concatenate([np.expand_dims(np.load(file), axis=0) for file in raw_data_files], axis=0)
    seq_data = np.concatenate([np.load(file) for file in seq_data_files], axis=0)

    max_seq_length = max(max_seq_length, seq_data.shape[0])  # Update the maximum sequence length

    X.append(raw_data)
    y.append(seq_data)

# Pad or truncate sequences to the maximum sequence length
X = np.concatenate([np.pad(data, ((0, max_seq_length - data.shape[0]), (0, 0)), mode='constant') for data in X], axis=0)
y = np.concatenate([np.pad(data, ((0, max_seq_length - data.shape[0]), (0, 0)), mode='constant') for data in y], axis=0)


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 144 and the array at index 1 has size 115

In [5]:
# Prepare the data
seq_length = 10
X = []
y = []

for action in actions:
    raw_data = np.load(f"{dataset_path}raw_{action}.npy")
    seq_data = np.load(f"{dataset_path}seq_{action}.npy")
    X.append(raw_data)
    y.append(seq_data)

X = np.concatenate(X, axis=0)
y = np.concatenate(y, axis=0)

FileNotFoundError: [Errno 2] No such file or directory: 'dataset/raw_action1.npy'

In [ ]:
# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Define the LSTM model
model = keras.Sequential([
    keras.layers.LSTM(64, return_sequences=True, input_shape=(X.shape[1], X.shape[2])),
    keras.layers.LSTM(64),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(seq_length * 3)  # Output layer with flattened landmarks (seq_length * 3 features)
])


In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='mse')

In [ ]:
# Train the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=64)


In [ ]:
# Save the trained model
model.save('trained_model.h5')